## Commands to run first:



cut -f 1,2 hg19.fa.fai > hg19.txt


bedtools makewindows -g hg19.txt -w 200000 > hg19.bed


bedtools getfasta -fi hg19.fa -bed hg19.bed > hg19.win.fa


faCount hg19.win.fa > hg19.facount.txt
 

In [2]:
import pandas as pd
import os

# facount_input_hg38 = "/g/korbel2/weber/workspace/mosaicatcher-update/workflow/data/ref_genomes/HG38/hg38.facount.txt"
# facount_input_hg19 = "/g/korbel2/weber/workspace/mosaicatcher-update/workflow/data/ref_genomes/HG19/hg19.facount.txt"
# facount_input_T2T = "/g/korbel2/weber/workspace/mosaicatcher-update/workflow/data/ref_genomes/T2T/T2T.facount.txt"
facount_input_MM10 = "/g/korbel2/weber/workspace/mosaicatcher-update/workflow/data/ref_genomes/MM10/MM10.facount.txt"
output_dir = "/g/korbel2/weber/workspace/mosaicatcher-update/workflow/data/GC/"
output_dir = "/g/korbel2/weber/workspace/ashleys-qc-pipeline/workflow/data/GC/"

# l_assemblies = [facount_input_hg38, facount_input_hg19, facount_input_T2T]
l_assemblies = [facount_input_MM10]


for facount_input in l_assemblies:
    print(facount_input)
    df = pd.read_csv(facount_input, sep="\t")
    df = df.loc[df["#seq"].str.contains("chr[0-9,X,Y]{1,2}:", regex=True)]
    df = df.astype({
        "A" : int,
        "C" : int,
        "G" : int,
        "T" : int,
        "N" : int,
        "cpg" : int,
    })
    df["%GC"] = 100 * ((df["C"] + df["G"]) / (df["A"] + df["T"] + df["C"] + df["G"]))
    df["cpg"] = df["cpg"].fillna("NA")
    df["%GC"] = df["%GC"].fillna("NA")

    df[["chrom", "start", "end"]] = df["#seq"].str.split(":|-", expand=True)
    df = df[["chrom", "start", "end", "A", "C", "G", "T", "N", "cpg", "%GC"]]

    df.to_csv(output_dir + os.path.basename(facount_input).replace("facount.txt", "GC_matrix.txt.gz"), sep="\t", index=False, compression="gzip")


/g/korbel2/weber/workspace/mosaicatcher-update/workflow/data/ref_genomes/MM10/MM10.facount.txt
